# Ottimizazzione della funzione ***frag_faces***

In [7]:
using Pkg
Pkg.build()

LoadError: `ViewerGL` is a direct dependency, but does not appear in the manifest. If you intend `ViewerGL` to be a direct dependency, run `Pkg.resolve()` to populate the manifest. Otherwise, remove `ViewerGL` with `Pkg.rm("ViewerGL")`. Finally, run `Pkg.instantiate()` again.

In [4]:
#Carico le librerie utili all'implementazione ed alla fase di test
using Profile
using ProfileView
using BenchmarkTools #utile per il test
using SparseArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
using NearestNeighbors
Lar = LinearAlgebraicRepresentation

LoadError: ArgumentError: Package LinearAlgebraicRepresentation [95167b0c-186a-594b-a231-52369c6de044] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


Testiamo la funzione merge_vertices originale

In [ ]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

Preparazione input

In [ ]:
V = [0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; 0.5941251 0.8769965 0.3624924; 0.6542998 0.8170598 1.0344647; 1.3260341 0.2707609 0.2428771; 1.3862088 0.2108241 0.9148494; 1.2660973 0.9422582 0.3081388; 1.326272 0.8823214 0.980111; -0.3874063 0.4902226 0.4536339; 0.3249123 0.707347 0.5231232; -0.1702819 -0.0864242 0.0297177; 0.5420367 0.1307001 0.099207; -0.317917 0.0663064 1.0658723; 0.3944016 0.2834308 1.1353616; -0.1007926 -0.5103404 0.6419561; 0.611526 -0.2932161 0.7114454; 0.7899026 0.0605793 0.6679889; 0.46601 0.0749997 0.6686316; 0.804323 0.3844725 0.6679746; 0.4804304 0.3988929 0.6686173; 0.7905452 0.060565 0.9922023; 0.4666527 0.0749854 0.992845; 0.8049656 0.3844582 0.992188; 0.4810731 0.3988786 0.9928307; -0.2261907 -0.0720455 0.4715635; -0.0499888 0.0863489 0.7965885; -0.0677963 0.219164 0.24378; 0.1084056 0.3775584 0.5688049; 0.0988343 -0.2998291 0.4063673; 0.2750362 -0.1414347 0.7313923; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087];

EV = SparseArrays.sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2, 6, 12, 3, 7, 9, 3, 8, 10, 4, 7, 11, 4, 8, 12, 13, 17, 21, 13, 18, 22, 14, 17, 23, 14, 18, 24, 15, 19, 21, 15, 20, 22, 16, 19, 23, 16, 20, 24, 25, 29, 33, 25, 30, 34, 26, 29, 35, 26, 30, 36, 27, 31, 33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1]);

FE = SparseArrays.sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5, 1, 6, 2, 5, 2, 6, 3, 5, 3, 6, 4, 5, 4, 6, 7, 9, 7, 10, 8, 9, 8, 10, 7, 11, 7, 12, 8, 11, 8, 12, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 13, 17, 13, 18, 14, 17, 14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1]);

Testiamo la funzione con l'utilizzo di @btime della libreria BenchmarkTools

In [ ]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

# Miglioramento prestazioni

In [ ]:
function frag_face_revisited(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, 
    sp_idx::Vector{Int64}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
    tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    @async for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

In [ ]:
@btime frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)

In [ ]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [ ]:
ProfileView.view()
Profile.print()